In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.types import StringType
from pyspark.sql.functions import col
from pyspark.sql.types import StringType, StructType, StructField


In [2]:
from pyspark.sql import SparkSession
spark = (SparkSession.builder
                .config("spark.jars.packages", "org.apache.spark:spark-sql-kafka-0-10_2.12:3.3.1,org.apache.iceberg:iceberg-spark-runtime-3.5_2.12:1.7.1,org.apache.hadoop:hadoop-aws:3.3.2")
                .config("spark.hadoop.fs.s3a.endpoint", "http://localhost:9000") 
                .config("spark.hadoop.fs.s3a.access.key", "minio") 
                .config("spark.hadoop.fs.s3a.secret.key", "minio123") 
                .config("spark.hadoop.fs.s3a.path.style.access", "true") 
                .config("spark.hadoop.fs.s3a.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem")
                .config("spark.sql.catalog.spark_catalog", "org.apache.iceberg.spark.SparkSessionCatalog")
                .config("spark.sql.catalog.spark_catalog.type", "hadoop")
                .config("spark.sql.catalog.spark_catalog.warehouse", "s3a://commerce/warehouse/") 
                .config("spark.sql.extensions", "org.apache.iceberg.spark.extensions.IcebergSparkSessionExtensions") 
                .appName("Kafka to predictions Pipeline")
         .getOrCreate())

25/02/13 13:20:47 WARN Utils: Your hostname, Gauravs-MacBook-Air-3.local resolves to a loopback address: 127.0.0.1; using 192.168.1.27 instead (on interface en0)
25/02/13 13:20:47 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


:: loading settings :: url = jar:file:/Users/saurabh/Desktop/workspace/realtime_fraud_detection_system/.venv/lib/python3.11/site-packages/pyspark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /Users/saurabh/.ivy2/cache
The jars for the packages stored in: /Users/saurabh/.ivy2/jars
org.apache.spark#spark-sql-kafka-0-10_2.12 added as a dependency
org.apache.iceberg#iceberg-spark-runtime-3.5_2.12 added as a dependency
org.apache.hadoop#hadoop-aws added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-9bbd1d88-cf54-4033-b880-fe1a2b3e02d1;1.0
	confs: [default]
	found org.apache.spark#spark-sql-kafka-0-10_2.12;3.3.1 in central
	found org.apache.spark#spark-token-provider-kafka-0-10_2.12;3.3.1 in central
	found org.apache.kafka#kafka-clients;2.8.1 in central
	found org.lz4#lz4-java;1.8.0 in central
	found org.xerial.snappy#snappy-java;1.1.8.4 in central
	found org.slf4j#slf4j-api;1.7.32 in central
	found org.apache.hadoop#hadoop-client-runtime;3.3.2 in central
	found org.spark-project.spark#unused;1.0.0 in central
	found org.apache.hadoop#hadoop-client-api;3.3.2 in central
	found commons-logging#commons-logging;1.1.3 in ce

In [3]:
def execute_query(query: str):
    spark.sql(query).show(truncate=False)

In [4]:
# Kafka configurations
kafka_brokers = "localhost:9093"  # Update with your Kafka broker address
kafka_topic = "debezium.payment.transactions"
# Read data from Kafka (batch mode)
kafka_df = spark.read \
    .format("kafka") \
    .option("kafka.bootstrap.servers", kafka_brokers) \
    .option("subscribe", kafka_topic) \
    .option("startingOffsets", "earliest") \
    .load()

In [5]:
kafka_df.select("value").show(truncate=False)

25/02/13 13:21:28 WARN MetricsConfig: Cannot locate configuration: tried hadoop-metrics2-s3a-file-system.properties,hadoop-metrics2.properties
25/02/13 13:21:29 WARN AdminClientConfig: The configuration 'key.deserializer' was supplied but isn't a known config.
25/02/13 13:21:29 WARN AdminClientConfig: The configuration 'value.deserializer' was supplied but isn't a known config.
25/02/13 13:21:29 WARN AdminClientConfig: The configuration 'enable.auto.commit' was supplied but isn't a known config.
25/02/13 13:21:29 WARN AdminClientConfig: The configuration 'max.poll.records' was supplied but isn't a known config.
25/02/13 13:21:29 WARN AdminClientConfig: The configuration 'auto.offset.reset' was supplied but isn't a known config.


+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [6]:
from pyspark.sql.types import StructType, StructField, IntegerType, DoubleType, StringType, LongType,BinaryType
from pyspark.sql.types import StructType, StructField, IntegerType, BinaryType, StringType, LongType, DecimalType
from pyspark.sql.functions import udf, col, from_json
from decimal import Decimal

# Decode bytes to Decimal
def decode_decimal(bytes_value):
    if bytes_value is None:
        return None
    scale = 2  # Defined in the Debezium schema
    precision = 10
    # Decode the bytes to integer and apply scale
    unscaled_value = int.from_bytes(bytes_value, byteorder="big", signed=True)
    return Decimal(unscaled_value) / (10 ** scale)

decode_decimal_udf=spark.udf.register("decode_decimal_udf",decode_decimal,  DecimalType(precision=10, scale=2))
# Register the UDF
#decode_decimal_udf = udf(decode_decimal, DecimalType(precision=10, scale=2))
# Define the schema for the JSON messages
root_schema = StructType([
    StructField("schema", StructType([
        StructField("type", StringType(), False),
        StructField("fields", StringType(), True),  # Simplified for nested schema
        StructField("optional", StringType(), True),
        StructField("name", StringType(), True),
        StructField("version", IntegerType(), True)
    ]), True),
    StructField("payload", StructType([
        StructField("before", StructType([
            StructField("tx_id", IntegerType(), True),
            StructField("tx_datetime", LongType(), True),
            StructField("customer_id", IntegerType(), True),
            StructField("terminal_id", IntegerType(), True),
            StructField("tx_amount", BinaryType(), True)  # Use StringType to handle Base64-encoded Decimal
        ]), True),
        StructField("after", StructType([
            StructField("tx_id", IntegerType(), True),
            StructField("tx_datetime", LongType(), True),
            StructField("customer_id", IntegerType(), True),
            StructField("terminal_id", IntegerType(), True),
            StructField("tx_amount", BinaryType(), True) 
        ]), True),
        StructField("source", StructType([
            StructField("version", StringType(), True),
            StructField("connector", StringType(), True),
            StructField("name", StringType(), True),
            StructField("ts_ms", LongType(), True),
            StructField("snapshot", StringType(), True),
            StructField("db", StringType(), True),
            StructField("sequence", StringType(), True),
            StructField("ts_us", LongType(), True),
            StructField("ts_ns", LongType(), True),
            StructField("schema", StringType(), True),
            StructField("table", StringType(), True),
            StructField("txId", LongType(), True),
            StructField("lsn", LongType(), True),
            StructField("xmin", LongType(), True)
        ]), True),
        StructField("transaction", StructType([
            StructField("id", StringType(), True),
            StructField("total_order", LongType(), True),
            StructField("data_collection_order", LongType(), True)
        ]), True),
        StructField("op", StringType(), True),
        StructField("ts_ms", LongType(), True),
        StructField("ts_us", LongType(), True),
        StructField("ts_ns", LongType(), True)
    ]))
])

from pyspark.sql.types import StructType,DecimalType, StructField, IntegerType, DoubleType, StringType, LongType,BinaryType

from pyspark.sql.functions import from_json

kafka_df = kafka_df.withColumn("value", col("value").cast(StringType())) \
    .withColumn("key", col("key").cast(StringType()))
# Apply the schema to the DataFrame
kafka_df = kafka_df.withColumn("value", from_json("value", root_schema))


# Create a temporary view for further SQL operations


kafka_df.createOrReplaceTempView("kafkadata")


In [7]:
execute_query("select * from kafkadata")

25/02/13 13:21:44 WARN AdminClientConfig: The configuration 'key.deserializer' was supplied but isn't a known config.
25/02/13 13:21:44 WARN AdminClientConfig: The configuration 'value.deserializer' was supplied but isn't a known config.
25/02/13 13:21:44 WARN AdminClientConfig: The configuration 'enable.auto.commit' was supplied but isn't a known config.
25/02/13 13:21:44 WARN AdminClientConfig: The configuration 'max.poll.records' was supplied but isn't a known config.
25/02/13 13:21:44 WARN AdminClientConfig: The configuration 'auto.offset.reset' was supplied but isn't a known config.


+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [8]:
execute_query(""" create or replace temp view latest_data as 
              with extracted_data  as (
                    select 
                    value.payload.after.tx_id,
                    to_timestamp(from_unixtime(value.payload.after.tx_datetime/1000000)) AS tx_datetime, -- Conversion from Unix to Timestamp
                    value.payload.after.customer_id,
                    value.payload.after.terminal_id,
                    decode_decimal_udf(value.payload.after.tx_amount) AS tx_amount, -- Conversion to DECIMAL(10,2)
                    timestamp
                    from kafkadata),
              ranked_data as (
                  select tx_id,
                tx_datetime,
                customer_id,
                terminal_id,
                tx_amount,
                  timestamp,
                  ROW_NUMBER() over (partition by tx_id order by timestamp DESC) as rn
                  from extracted_data) 
                select tx_id,
                    tx_datetime,
                    customer_id,
                    terminal_id,
                    tx_amount,
                  now() as row_created_timestamp,
                  now() as row_updated_timestamp
              from ranked_data 
              where rn=1""")

++
||
++
++



In [9]:
execute_query('select * from latest_data')

25/02/13 13:22:02 WARN AdminClientConfig: The configuration 'key.deserializer' was supplied but isn't a known config.
25/02/13 13:22:02 WARN AdminClientConfig: The configuration 'value.deserializer' was supplied but isn't a known config.
25/02/13 13:22:02 WARN AdminClientConfig: The configuration 'enable.auto.commit' was supplied but isn't a known config.
25/02/13 13:22:02 WARN AdminClientConfig: The configuration 'max.poll.records' was supplied but isn't a known config.
25/02/13 13:22:02 WARN AdminClientConfig: The configuration 'auto.offset.reset' was supplied but isn't a known config.
25/02/13 13:22:03 WARN KafkaDataConsumer: KafkaDataConsumer is not running in UninterruptibleThread. It may hang when KafkaDataConsumer's methods are interrupted because of KAFKA-1894
25/02/13 13:22:04 WARN KafkaDataConsumer: KafkaDataConsumer is not running in UninterruptibleThread. It may hang when KafkaDataConsumer's methods are interrupted because of KAFKA-1894
25/02/13 13:22:04 WARN KafkaDataConsu

+-----+-------------------+-----------+-----------+---------+--------------------------+--------------------------+
|tx_id|tx_datetime        |customer_id|terminal_id|tx_amount|row_created_timestamp     |row_updated_timestamp     |
+-----+-------------------+-----------+-----------+---------+--------------------------+--------------------------+
|0    |2025-01-15 17:02:19|3          |2          |120.50   |2025-02-13 13:22:01.961522|2025-02-13 13:22:01.961522|
|1    |2025-01-28 14:42:45|7          |4          |897.34   |2025-02-13 13:22:01.961522|2025-02-13 13:22:01.961522|
|2    |2025-01-20 20:15:50|1          |1          |543.78   |2025-02-13 13:22:01.961522|2025-02-13 13:22:01.961522|
|3    |2025-01-19 00:02:10|0          |3          |67.89    |2025-02-13 13:22:01.961522|2025-02-13 13:22:01.961522|
|4    |2025-01-29 10:53:40|4          |0          |299.99   |2025-02-13 13:22:01.961522|2025-02-13 13:22:01.961522|
|5    |2025-01-27 16:15:00|9          |1          |876.21   |2025-02-13 

## predictions


In [4]:
from pyspark.sql import SparkSession
import pickle

# Initialize SparkSession
spark = SparkSession.builder \
    .appName("FraudDetection") \
    .getOrCreate()

# Load the scikit-learn model from a pickle file
with open("random_forest_model.pkl", "rb") as file:
    sklearn_model = pickle.load(file)

# Broadcast the model using SparkContext
broadcast_model = spark.sparkContext.broadcast(sklearn_model)

# You can now use `broadcast_model.value` to access the broadcasted model
print("Model broadcasted successfully.")


Model broadcasted successfully.


In [ ]:
# Define a UDF for predictions
from pyspark.sql.types import FloatType
def predict(features):
    model = broadcast_model.value
    # Convert the list of features to the expected shape for the model
    prediction = model.predict([features])
    return float(prediction[0])

# Register the UDF
predict_udf = udf(predict, FloatType())

# Apply the UDF to add predictions to the DataFrame
df_with_predictions = df.withColumn("prediction", predict_udf(df["features"]))

# Show the result
df_with_predictions.show()

In [32]:
execute_query("CREATE DATABASE IF NOT EXISTS payment;")
execute_query("show databases")

++
||
++
++

+---------+
|namespace|
+---------+
|default  |
|payment  |
+---------+



In [33]:
execute_query("""
      CREATE TABLE IF NOT EXISTS payment.transactions (
                    tx_id INT,
                    tx_datetime TIMESTAMP,
                    customer_id INT,
                    terminal_id INT,
                    tx_amount DECIMAL(10,2),
    row_created_timestamp TIMESTAMP,
    row_updated_timestamp TIMESTAMP
)
USING iceberg
""")

++
||
++
++



In [34]:
execute_query("describe latest_data")

+---------------------+-------------+-------+
|col_name             |data_type    |comment|
+---------------------+-------------+-------+
|tx_id                |int          |NULL   |
|tx_datetime          |timestamp    |NULL   |
|customer_id          |int          |NULL   |
|terminal_id          |int          |NULL   |
|tx_amount            |decimal(10,2)|NULL   |
|row_created_timestamp|timestamp    |NULL   |
|row_updated_timestamp|timestamp    |NULL   |
+---------------------+-------------+-------+



In [37]:
#insert data
execute_query("""
MERGE INTO payment.transactions AS target
USING latest_data AS source
ON target.tx_id = source.tx_id
WHEN MATCHED THEN
  UPDATE SET
    target.tx_datetime = source.tx_datetime,
    target.customer_id = source.customer_id,
    target.terminal_id = source.terminal_id,
   target.tx_amount = source.tx_amount,      
   target.row_updated_timestamp = source.row_updated_timestamp
WHEN NOT MATCHED THEN
  INSERT (          tx_id,
                    tx_datetime,
                    customer_id,
                    terminal_id,
                    tx_amount,
    row_created_timestamp,
    row_updated_timestamp
  )
  VALUES (
    source.tx_id,
    source.tx_datetime,
    source.customer_id,
    source.terminal_id,
    source.tx_amount,
    source.row_created_timestamp,
    source.row_updated_timestamp
  )
""")

25/02/12 23:13:10 WARN AdminClientConfig: The configuration 'key.deserializer' was supplied but isn't a known config.
25/02/12 23:13:10 WARN AdminClientConfig: The configuration 'value.deserializer' was supplied but isn't a known config.
25/02/12 23:13:10 WARN AdminClientConfig: The configuration 'enable.auto.commit' was supplied but isn't a known config.
25/02/12 23:13:10 WARN AdminClientConfig: The configuration 'max.poll.records' was supplied but isn't a known config.
25/02/12 23:13:10 WARN AdminClientConfig: The configuration 'auto.offset.reset' was supplied but isn't a known config.
25/02/12 23:13:10 WARN AdminClientConfig: The configuration 'key.deserializer' was supplied but isn't a known config.
25/02/12 23:13:10 WARN AdminClientConfig: The configuration 'value.deserializer' was supplied but isn't a known config.
25/02/12 23:13:10 WARN AdminClientConfig: The configuration 'enable.auto.commit' was supplied but isn't a known config.
25/02/12 23:13:10 WARN AdminClientConfig: The c

++
||
++
++



In [38]:
execute_query("select * from payment.transactions")

+-----+-------------------+-----------+-----------+---------+--------------------------+--------------------------+
|tx_id|tx_datetime        |customer_id|terminal_id|tx_amount|row_created_timestamp     |row_updated_timestamp     |
+-----+-------------------+-----------+-----------+---------+--------------------------+--------------------------+
|0    |2025-01-15 17:02:19|3          |2          |120.50   |2025-02-12 22:31:54.247392|2025-02-12 23:13:09.022766|
|1    |2025-01-28 14:42:45|7          |4          |897.34   |2025-02-12 22:31:54.247392|2025-02-12 23:13:09.022766|
|2    |2025-01-20 20:15:50|1          |1          |543.78   |2025-02-12 22:31:54.247392|2025-02-12 23:13:09.022766|
|3    |2025-01-19 00:02:10|0          |3          |67.89    |2025-02-12 22:31:54.247392|2025-02-12 23:13:09.022766|
|4    |2025-01-29 10:53:40|4          |0          |299.99   |2025-02-12 22:31:54.247392|2025-02-12 23:13:09.022766|
|5    |2025-01-27 16:15:00|9          |1          |876.21   |2025-02-12 

## StreamingJob

In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.types import StringType
from pyspark.sql.functions import col
from pyspark.sql.types import StringType, StructType, StructField

In [2]:
KAFKA_BROKERS = 'localhost:9093'
#KAFKA_TOPIC_NAME = "debezium.payment.transactions"
S3_ENDPOINT = "http://localhost:9000"
CHECKPOINT_PATH = "s3a://commerce/checkpoints/debezium.payment.transactions/"

In [3]:
from pyspark.sql import SparkSession
spark = (SparkSession.builder
                .config("spark.jars.packages", "org.apache.spark:spark-sql-kafka-0-10_2.12:3.3.1,org.apache.iceberg:iceberg-spark-runtime-3.5_2.12:1.7.1,org.apache.hadoop:hadoop-aws:3.3.2")
                .config("spark.hadoop.fs.s3a.endpoint", S3_ENDPOINT) 
                .config("spark.hadoop.fs.s3a.access.key", "minio") 
                .config("spark.hadoop.fs.s3a.secret.key", "minio123") 
                .config("spark.hadoop.fs.s3a.path.style.access", "true") 
                .config("spark.hadoop.fs.s3a.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem")
                .config("spark.sql.catalog.spark_catalog", "org.apache.iceberg.spark.SparkSessionCatalog")
                .config("spark.sql.catalog.spark_catalog.type", "hadoop")
                .config("spark.sql.catalog.spark_catalog.warehouse", "s3a://commerce/warehouse/") 
                .config("spark.sql.extensions", "org.apache.iceberg.spark.extensions.IcebergSparkSessionExtensions") 
                .appName("streaming_sink_transactions")
         .getOrCreate())

25/02/12 23:17:50 WARN Utils: Your hostname, Gauravs-MacBook-Air-3.local resolves to a loopback address: 127.0.0.1; using 192.168.1.27 instead (on interface en0)
25/02/12 23:17:50 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


:: loading settings :: url = jar:file:/Users/saurabh/Desktop/workspace/realtime_fraud_detection_system/.venv/lib/python3.11/site-packages/pyspark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /Users/saurabh/.ivy2/cache
The jars for the packages stored in: /Users/saurabh/.ivy2/jars
org.apache.spark#spark-sql-kafka-0-10_2.12 added as a dependency
org.apache.iceberg#iceberg-spark-runtime-3.5_2.12 added as a dependency
org.apache.hadoop#hadoop-aws added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-5dcefc09-d6e5-48d9-96fe-ee4b52b5a3f6;1.0
	confs: [default]
	found org.apache.spark#spark-sql-kafka-0-10_2.12;3.3.1 in central
	found org.apache.spark#spark-token-provider-kafka-0-10_2.12;3.3.1 in central
	found org.apache.kafka#kafka-clients;2.8.1 in central
	found org.lz4#lz4-java;1.8.0 in central
	found org.xerial.snappy#snappy-java;1.1.8.4 in central
	found org.slf4j#slf4j-api;1.7.32 in central
	found org.apache.hadoop#hadoop-client-runtime;3.3.2 in central
	found org.spark-project.spark#unused;1.0.0 in central
	found org.apache.hadoop#hadoop-client-api;3.3.2 in central
	found commons-logging#commons-logging;1.1.3 in ce

----------------------------------------
Exception occurred during processing of request from ('127.0.0.1', 60011)
Traceback (most recent call last):
  File "/opt/homebrew/Cellar/python@3.11/3.11.10/Frameworks/Python.framework/Versions/3.11/lib/python3.11/socketserver.py", line 317, in _handle_request_noblock
    self.process_request(request, client_address)
  File "/opt/homebrew/Cellar/python@3.11/3.11.10/Frameworks/Python.framework/Versions/3.11/lib/python3.11/socketserver.py", line 348, in process_request
    self.finish_request(request, client_address)
  File "/opt/homebrew/Cellar/python@3.11/3.11.10/Frameworks/Python.framework/Versions/3.11/lib/python3.11/socketserver.py", line 361, in finish_request
    self.RequestHandlerClass(request, client_address, self)
  File "/opt/homebrew/Cellar/python@3.11/3.11.10/Frameworks/Python.framework/Versions/3.11/lib/python3.11/socketserver.py", line 755, in __init__
    self.handle()
  File "/Users/saurabh/Desktop/workspace/realtime_fraud_detec

In [4]:
# Read data from Kafka (batch mode)
kafka_df = spark.readStream \
    .format("kafka") \
    .option("kafka.bootstrap.servers", KAFKA_BROKERS) \
    .option("subscribe", "debezium.payment.transactions") \
    .option("startingOffsets", "earliest") \
    .load()

In [5]:
from pyspark.sql.types import StructType, StructField, IntegerType, DoubleType, StringType, LongType,BinaryType
from pyspark.sql.types import StructType, StructField, IntegerType, BinaryType, StringType, LongType, DecimalType
from pyspark.sql.functions import udf, col, from_json
from decimal import Decimal

# Decode bytes to Decimal
def decode_decimal(bytes_value):
    if bytes_value is None:
        return None
    scale = 2  # Defined in the Debezium schema
    precision = 10
    # Decode the bytes to integer and apply scale
    unscaled_value = int.from_bytes(bytes_value, byteorder="big", signed=True)
    return Decimal(unscaled_value) / (10 ** scale)

decode_decimal_udf=spark.udf.register("decode_decimal_udf",decode_decimal,  DecimalType(precision=10, scale=2))
# Register the UDF
#decode_decimal_udf = udf(decode_decimal, DecimalType(precision=10, scale=2))
# Define the schema for the JSON messages
root_schema = StructType([
    StructField("schema", StructType([
        StructField("type", StringType(), False),
        StructField("fields", StringType(), True),  # Simplified for nested schema
        StructField("optional", StringType(), True),
        StructField("name", StringType(), True),
        StructField("version", IntegerType(), True)
    ]), True),
    StructField("payload", StructType([
        StructField("before", StructType([
            StructField("tx_id", IntegerType(), True),
            StructField("tx_datetime", LongType(), True),
            StructField("customer_id", IntegerType(), True),
            StructField("terminal_id", IntegerType(), True),
            StructField("tx_amount", BinaryType(), True)  # Use StringType to handle Base64-encoded Decimal
        ]), True),
        StructField("after", StructType([
            StructField("tx_id", IntegerType(), True),
            StructField("tx_datetime", LongType(), True),
            StructField("customer_id", IntegerType(), True),
            StructField("terminal_id", IntegerType(), True),
            StructField("tx_amount", BinaryType(), True) 
        ]), True),
        StructField("source", StructType([
            StructField("version", StringType(), True),
            StructField("connector", StringType(), True),
            StructField("name", StringType(), True),
            StructField("ts_ms", LongType(), True),
            StructField("snapshot", StringType(), True),
            StructField("db", StringType(), True),
            StructField("sequence", StringType(), True),
            StructField("ts_us", LongType(), True),
            StructField("ts_ns", LongType(), True),
            StructField("schema", StringType(), True),
            StructField("table", StringType(), True),
            StructField("txId", LongType(), True),
            StructField("lsn", LongType(), True),
            StructField("xmin", LongType(), True)
        ]), True),
        StructField("transaction", StructType([
            StructField("id", StringType(), True),
            StructField("total_order", LongType(), True),
            StructField("data_collection_order", LongType(), True)
        ]), True),
        StructField("op", StringType(), True),
        StructField("ts_ms", LongType(), True),
        StructField("ts_us", LongType(), True),
        StructField("ts_ns", LongType(), True)
    ]))
])

from pyspark.sql.types import StructType,DecimalType, StructField, IntegerType, DoubleType, StringType, LongType,BinaryType

from pyspark.sql.functions import from_json

kafka_df = kafka_df.withColumn("value", col("value").cast(StringType())) \
    .withColumn("key", col("key").cast(StringType()))
# Apply the schema to the DataFrame
kafka_df = kafka_df.withColumn("value", from_json("value", root_schema))


# Create a temporary view for further SQL operations


kafka_df.createOrReplaceTempView("kafkadata")


25/02/12 23:18:01 WARN MetricsConfig: Cannot locate configuration: tried hadoop-metrics2-s3a-file-system.properties,hadoop-metrics2.properties


In [6]:
spark.sql("""
create database if not exists payment
""").show(truncate=False)

++
||
++
++



In [7]:
spark.sql("""
      CREATE TABLE IF NOT EXISTS payment.transactions (
                    tx_id INT,
                    tx_datetime Timestamp,
                    customer_id INT,
                    terminal_id INT,
                    tx_amount DECIMAL(10,2),
    row_created_timestamp TIMESTAMP,
    row_updated_timestamp TIMESTAMP
)
USING iceberg
""")

DataFrame[]

In [8]:
def foreach_batch_function(df, epoch_id):
    spark = df.sparkSession
    df.createOrReplaceTempView('micro_batch')
    latest_data = spark.sql(""" 
              with extracted_data  as (
                    select 
                    value.payload.after.tx_id,
                    to_timestamp(from_unixtime(value.payload.after.tx_datetime/1000000)) AS tx_datetime, -- Conversion from Unix to Timestamp
                    value.payload.after.customer_id,
                    value.payload.after.terminal_id,
                     decode_decimal_udf(value.payload.after.tx_amount) AS tx_amount, -- Conversion to DECIMAL(10,2)
                    timestamp
                    from micro_batch),
              ranked_data as (
                  select tx_id,
                tx_datetime,
                customer_id,
                terminal_id,
                tx_amount,
                  timestamp,
                  ROW_NUMBER() over (partition by tx_id order by timestamp DESC) as rn
                  from extracted_data) 
                select tx_id,
                    tx_datetime,
                    customer_id,
                    terminal_id,
                    tx_amount,
                  now() as row_created_timestamp,
                  now() as row_updated_timestamp
              from ranked_data 
              where rn=1""")
    latest_data.createOrReplaceTempView('latest_data')
    #insert data
    spark.sql("""
    MERGE INTO payment.transactions AS target
    USING latest_data AS source
ON target.tx_id = source.tx_id
WHEN MATCHED THEN
  UPDATE SET
    target.tx_datetime = source.tx_datetime,
    target.customer_id = source.customer_id,
    target.terminal_id = source.terminal_id,
    target.tx_amount = source.tx_amount,      
    target.row_updated_timestamp = source.row_updated_timestamp
WHEN NOT MATCHED THEN
  INSERT (          tx_id,
                    tx_datetime,
                    customer_id,
                    terminal_id,
                    tx_amount,
    row_created_timestamp,
    row_updated_timestamp
  )
  VALUES (
    source.tx_id,
    source.tx_datetime,
    source.customer_id,
    source.terminal_id,
    source.tx_amount,
    source.row_created_timestamp,
    source.row_updated_timestamp
  )
""").show(truncate=False)
    

query = kafka_df.writeStream \
.foreachBatch(foreach_batch_function) \
.option("checkpointLocation", CHECKPOINT_PATH) \
.trigger(processingTime='30 seconds') \
.start()
    

25/02/12 23:18:07 WARN ResolveWriteToStream: spark.sql.adaptive.enabled is not supported in streaming DataFrames/Datasets and will be disabled.


In [9]:
query.awaitTermination()

25/02/12 23:18:27 WARN KafkaDataConsumer: KafkaDataConsumer is not running in UninterruptibleThread. It may hang when KafkaDataConsumer's methods are interrupted because of KAFKA-1894
25/02/12 23:18:27 WARN KafkaDataConsumer: KafkaDataConsumer is not running in UninterruptibleThread. It may hang when KafkaDataConsumer's methods are interrupted because of KAFKA-1894
25/02/12 23:18:27 WARN KafkaDataConsumer: KafkaDataConsumer is not running in UninterruptibleThread. It may hang when KafkaDataConsumer's methods are interrupted because of KAFKA-1894
25/02/12 23:18:27 WARN KafkaDataConsumer: KafkaDataConsumer is not running in UninterruptibleThread. It may hang when KafkaDataConsumer's methods are interrupted because of KAFKA-1894
25/02/12 23:18:27 WARN KafkaDataConsumer: KafkaDataConsumer is not running in UninterruptibleThread. It may hang when KafkaDataConsumer's methods are interrupted because of KAFKA-1894
25/02/12 23:18:27 WARN KafkaDataConsumer: KafkaDataConsumer is not running in Un

++
||
++
++



KeyboardInterrupt: 

25/02/12 23:51:04 WARN NetworkClient: [AdminClient clientId=adminclient-1] Connection to node 1001 (localhost/127.0.0.1:9093) could not be established. Broker may not be available.
25/02/12 23:51:04 WARN NetworkClient: [AdminClient clientId=adminclient-1] Connection to node 1001 (localhost/127.0.0.1:9093) could not be established. Broker may not be available.
25/02/12 23:51:05 WARN NetworkClient: [AdminClient clientId=adminclient-1] Connection to node 1001 (localhost/127.0.0.1:9093) could not be established. Broker may not be available.
25/02/12 23:51:05 WARN NetworkClient: [AdminClient clientId=adminclient-1] Connection to node 1001 (localhost/127.0.0.1:9093) could not be established. Broker may not be available.
25/02/12 23:51:06 WARN NetworkClient: [AdminClient clientId=adminclient-1] Connection to node 1001 (localhost/127.0.0.1:9093) could not be established. Broker may not be available.
25/02/12 23:51:07 WARN NetworkClient: [AdminClient clientId=adminclient-1] Connection to node 1